In [224]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import sklearn as sk 

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Data exploration

In [300]:
# apple mobility data
apple_mobility = pd.read_csv("https://covid19-static.cdn-apple.com/covid19-mobility-data/2204HotfixDev11/v3/en-us/applemobilitytrends-2021-12-17.csv")

/Users/jessbunnag/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [299]:
# get data 
factors = pd.read_csv('datasets/covid-state-data-agg/COVID19_state.csv')
covid_cases = pd.read_csv('datasets/covid-cases-data/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv')
state_trips = pd.read_csv('datasets/bts-travel-data/State_trips.csv')
blue_red = pd.read_csv('datasets/BlueRedStates.csv').drop(columns=['BlueRed'])

factors

,State,Tested,Infected,Deaths,Population,Pop Density,Gini,ICU Beds,Income,GDP,...,Hospitals,Health Spending,Pollution,Med-Large Airports,Temperature,Urban,Age 0-25,Age 26-54,Age 55+,School Closure Date
0,Alaska,620170,17057,84,734002,1.2863,0.4081,119,59687,73205,...,21,11064,6.4,1.0,26.60,66.0,0.36,0.39,0.25,03/19/20
1,Alabama,1356420,194892,2973,4908621,96.9221,0.4847,1533,42334,45219,...,101,7281,8.1,1.0,62.80,59.0,0.33,0.37,0.31,03/16/20
2,Arkansas,1363429,113641,1985,3038999,58.4030,0.4719,732,42566,42454,...,88,7408,7.1,0.0,60.40,56.2,0.34,0.37,0.30,03/17/20
3,Arizona,1792602,248139,5982,7378494,64.9550,0.4713,1559,43650,48055,...,83,6452,9.7,1.0,60.30,89.8,0.33,0.36,0.30,03/16/20
4,California,18912501,930628,17672,39937489,256.3727,0.4899,7338,62586,74205,...,359,7549,12.8,9.0,59.40,95.0,0.33,0.40,0.26,03/19/20
5,Colorado,2034795,109910,2105,5845526,56.4011,0.4586,1597,56846,63882,...,89,6804,6.7,1.0,45.10,86.2,0.33,0.40,0.27,03/23/20
6,Connecticut,2367994,73858,4627,3563077,735.8689,0.4945,674,74561,76342,...,32,9859,7.2,1.0,49.00,88.0,0.30,0.38,0.32,03/17/20
7,District of Columbia,528174,17438,647,720687,11814.5410,0.5420,314,47285,200277,...,10,11944,9.8,0.0,54.65,100.0,0.30,0.48,0.22,03/16/20
8,Delaware,565983,25311,710,982895,504.3073,0.4522,186,51449,77253,...,7,10254,8.3,0.0,55.30,83.3,0.30,0.37,0.33,03/16/20
9,Florida,10126764,801371,17043,21992985,410.1256,0.4852,5604,49417,48318,...,217,8076,7.4,7.0,70.70,91.2,0.29,0.37,0.35,03/16/20


In [293]:
# filter date and format date col 
ny_cols = ['NY', 'NYC']

def filterByYear(df, date_col, year):
    df[date_col] = pd.to_datetime(df[date_col], infer_datetime_format=True)
    return df[df[date_col].dt.strftime('%Y') == year]

covid_cases_cols = ['submission_date', 'state', 'new_case']
covid_cases = covid_cases[covid_cases_cols]
covid_cases = filterByYear(covid_cases, 'submission_date', '2020')

covid_cases[(covid_cases['submission_date'] == '2020-12-31')].sort_values('state')

,submission_date,state,new_case
964,2020-12-31,AK,372.0
26793,2020-12-31,AL,4784.0
2191,2020-12-31,AR,2708.0
38784,2020-12-31,AS,0.0
17831,2020-12-31,AZ,7721.0
14904,2020-12-31,CA,13410.0
33652,2020-12-31,CO,2858.0
15224,2020-12-31,CT,2045.0
29404,2020-12-31,DC,225.0
10103,2020-12-31,DE,626.0


In [294]:
# WAY1: merge NY and NYC data
cases_indexed = covid_cases.groupby(['submission_date','state']).sum()

indexes = cases_indexed.index
date_index = np.unique(np.array([ind[0] for ind in indexes]))

for date_ind in date_index: 
    cases_indexed.loc[date_ind, 'NY'] = sum(cases_indexed.loc[date_ind, 'NY'], cases_indexed.loc[date_ind, 'NYC'])

cases_indexed.reset_index(inplace=True, level=['submission_date', 'state'])
cases_indexed[(cases_indexed['submission_date'] == '2020-12-31') & (cases_indexed['state'].isin(ny_cols))]
# cases_indexed

new_cases = cases_indexed
new_cases[(new_cases['submission_date'] == '2020-12-31')].sort_values('state')

,submission_date,state,new_case
20640,2020-12-31,AK,372.0
20641,2020-12-31,AL,4784.0
20642,2020-12-31,AR,2708.0
20643,2020-12-31,AS,0.0
20644,2020-12-31,AZ,7721.0
20645,2020-12-31,CA,13410.0
20646,2020-12-31,CO,2858.0
20647,2020-12-31,CT,2045.0
20648,2020-12-31,DC,225.0
20649,2020-12-31,DE,626.0


In [277]:
# # WAY2: merge NYC with NY data
# covid_cases[(covid_cases['state'] == 'NY') | (covid_cases['state'] == 'NYC')].sort_values(['submission_date', 'state'])

# filt_NY = (covid_cases['state'] == 'NY') | (covid_cases['state'] == 'NYC')
# ny_cases = covid_cases.loc[filt_NY].groupby('submission_date').sum()\
#                     .reset_index().rename(columns={'new_case':'ny_cases'})

# merged = covid_cases.merge(ny_cases, on='submission_date', how='left')
# merged.sort_values(['submission_date', 'state'])

# merged[(merged['submission_date'] == '2020-12-31')].sort_values(['submission_date', 'state'])

# merged['state'] = np.where(merged['state'] == 'NY', merged['ny_cases'], merged['new_case'])
# merged = merged.drop(columns='ny_cases')

# merged[(merged['submission_date'] == '2020-12-31') ]

In [292]:
# state trips data 
state_trips_col = ['Date', 
                   'State Postal Code', 
                   'Number of Trips', 
                   'Number of Trips 5-10', 
                   'Number of Trips 25-50', 
                   'Number of Trips 100-250', 
                   'Number of Trips 250-500']
state_trips = state_trips[state_trips_col]
state_trips = filterByYear(state_trips, 'Date', '2020')

# check state num
state_trips.groupby(['Date']).size().reset_index(name='counts')

state_trips.head()

,Date,State Postal Code,Number of Trips,Number of Trips 5-10,Number of Trips 25-50,Number of Trips 100-250,Number of Trips 250-500
18615,2020-01-01,MS,"7,629,798","1,221,867","396,415","93,571","22,897"
18616,2020-01-01,CA,"112,606,885","15,443,353","5,553,845","839,553","174,752"
18617,2020-01-01,CO,"18,346,137","2,712,409","807,692","141,931","39,387"
18618,2020-01-01,CT,"9,636,627","1,491,090","464,270","49,982","10,008"
18619,2020-01-01,DE,"2,665,377","432,958","140,146","15,988","1,121"


In [297]:
new_cases.rename(columns={'submission_date' : 'date'}, inplace=True)
state_trips.rename(columns={'Date' : 'date', 'State Postal Code' : 'state'}, inplace=True)

# get population by state
factors = factors[['State', 'Population']]
pop_by_state = blue_red.merge(factors, left_on='stateName', right_on='State')\
    .drop(columns=['stateName', 'State'])\
    .rename(columns={'stateCode' : 'state'})

# merge all dfs
new_case_trips = new_cases.merge(state_trips, on=['date', 'state'], how='inner')
final_data = new_case_trips.merge(pop_by_state, on='state', how='inner')
final_data.replace(',','', regex=True, inplace=True)


# normalize number of trips 
num_trips_cols = ['Number of Trips', 'Number of Trips 5-10', 'Number of Trips 25-50', 'Number of Trips 100-250', 'Number of Trips 250-500']
for col in num_trips_cols:
    final_data[col] = final_data[col].astype(float) / final_data['Population']

# normalize num of new cases
final_data['new_case'] = final_data['new_case'] / final_data['Population']

# drop 0 new cases 
# index_to_drop = final_data[final_data['new_case'] == 0].index
# final_data.drop(index_to_drop, inplace=True)

final_data[final_data['new_case'] == 0].count

<bound method DataFrame.count of             date state  new_case  Number of Trips  Number of Trips 5-10  \
0     2020-01-22    AK       0.0         4.018937              0.561167   
1     2020-01-23    AK       0.0         3.814015              0.540654   
2     2020-01-24    AK       0.0         3.904329              0.563700   
3     2020-01-25    AK       0.0         3.388376              0.506086   
4     2020-01-26    AK       0.0         3.126530              0.443621   
...          ...   ...       ...              ...                   ...   
16965 2020-03-22    WY       0.0         2.748133              0.243425   
17134 2020-09-07    WY       0.0         2.662851              0.277637   
17214 2020-11-26    WY       0.0         2.452647              0.270679   
17242 2020-12-24    WY       0.0         2.710873              0.269916   
17243 2020-12-25    WY       0.0         1.626549              0.192577   

       Number of Trips 25-50  Number of Trips 100-250  \
0        

In [298]:
corr_matrix = final_data.corr()

corr_matrix

,new_case,Number of Trips,Number of Trips 5-10,Number of Trips 25-50,Number of Trips 100-250,Number of Trips 250-500,Population
new_case,1.000000,-0.205568,-0.182217,-0.011791,0.083361,0.084669,-0.030257
Number of Trips,-0.205568,1.000000,0.867849,0.602418,0.109070,0.047523,-0.100724
Number of Trips 5-10,-0.182217,0.867849,1.000000,0.489367,-0.067100,-0.020159,-0.013639
Number of Trips 25-50,-0.011791,0.602418,0.489367,1.000000,0.354798,0.159573,-0.135838
Number of Trips 100-250,0.083361,0.109070,-0.067100,0.354798,1.000000,0.781042,-0.199774
Number of Trips 250-500,0.084669,0.047523,-0.020159,0.159573,0.781042,1.000000,-0.112498
Population,-0.030257,-0.100724,-0.013639,-0.135838,-0.199774,-0.112498,1.000000


In [269]:
num_trips = np.array(final_data['Number of Trips'].astype(float)).reshape(-1, 1)
num_new_cases = final_data['new_case']

X_train, X_test, y_train, y_test = train_test_split(num_trips, num_new_cases, test_size=0.20)

model = LinearRegression()
model.fit(X_train, y_train)

# make a prediction
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

# find error
training_error = mean_squared_error(y_train, y_train_hat)
testing_error = mean_squared_error(y_test, y_test_hat)

print(training_error)
print(testing_error)

5755589.044854301
6641170.457705222


,new_case,Number of Trips,Number of Trips 5-10,Number of Trips 25-50,Number of Trips 100-250,Number of Trips 250-500,Population
new_case,1.000000,-0.189180,-0.113192,-0.068275,-0.081053,-0.044651,0.482867
Number of Trips,-0.189180,1.000000,0.867849,0.602418,0.109070,0.047523,-0.100724
Number of Trips 5-10,-0.113192,0.867849,1.000000,0.489367,-0.067100,-0.020159,-0.013639
Number of Trips 25-50,-0.068275,0.602418,0.489367,1.000000,0.354798,0.159573,-0.135838
Number of Trips 100-250,-0.081053,0.109070,-0.067100,0.354798,1.000000,0.781042,-0.199774
Number of Trips 250-500,-0.044651,0.047523,-0.020159,0.159573,0.781042,1.000000,-0.112498
Population,0.482867,-0.100724,-0.013639,-0.135838,-0.199774,-0.112498,1.000000


Apply mobility data

In [302]:
apple_mobility.head()

,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,...,2021-12-08,2021-12-09,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17
0,country/region,Albania,driving,NaN,NaN,NaN,100.0,95.30,101.43,97.20,...,171.58,159.52,173.82,170.79,154.11,156.63,145.56,151.96,160.56,178.86
1,country/region,Albania,walking,NaN,NaN,NaN,100.0,100.68,98.93,98.46,...,244.65,203.88,221.58,189.47,159.06,233.01,216.16,214.03,214.63,239.49
2,country/region,Argentina,driving,NaN,NaN,NaN,100.0,97.07,102.45,111.21,...,98.74,115.38,142.73,153.72,89.45,100.86,105.01,115.63,125.54,155.04
3,country/region,Argentina,walking,NaN,NaN,NaN,100.0,95.11,101.37,112.67,...,84.15,114.29,135.13,125.15,67.77,104.04,107.03,118.26,124.31,138.56
4,country/region,Australia,driving,AU,NaN,NaN,100.0,102.98,104.21,108.63,...,138.88,146.07,142.32,119.83,130.54,133.84,141.14,143.24,151.64,138.80


In [307]:
apple_mobility[apple_mobility['country'] == "United States"]

,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,...,2021-12-08,2021-12-09,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17
158,city,Akron,driving,NaN,Ohio,United States,100.0,103.06,107.50,106.14,...,146.38,154.32,179.24,167.41,121.65,142.76,152.04,155.13,162.09,180.14
159,city,Akron,transit,NaN,Ohio,United States,100.0,106.69,103.75,100.22,...,65.00,68.68,72.65,61.03,49.85,71.32,70.22,65.44,62.72,66.47
160,city,Akron,walking,NaN,Ohio,United States,100.0,97.23,79.05,74.77,...,118.31,122.80,145.72,136.49,99.43,112.08,113.91,121.07,128.97,140.42
161,city,Albany,driving,NaN,New York,United States,100.0,102.35,107.35,105.54,...,138.98,146.57,166.43,154.37,118.06,138.81,147.73,147.86,154.41,178.21
162,city,Albany,transit,NaN,New York,United States,100.0,100.14,105.95,107.76,...,146.68,149.19,159.92,155.78,113.66,135.25,137.81,131.96,133.02,137.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686,county,York County,walking,NaN,Pennsylvania,United States,100.0,110.22,121.82,109.72,...,191.71,197.18,223.87,209.56,167.35,184.86,196.13,209.28,211.66,250.22
4687,county,Young County,driving,NaN,Texas,United States,100.0,98.85,97.76,92.21,...,176.96,179.00,244.99,176.08,139.50,154.95,163.01,166.46,166.73,221.07
4688,county,Yuba County,driving,NaN,California,United States,100.0,102.98,103.10,114.08,...,169.29,168.30,191.33,177.60,121.76,142.89,148.62,159.91,169.67,198.52
4689,county,Yuma County,driving,NaN,Arizona,United States,100.0,101.82,106.67,108.93,...,157.59,160.94,188.79,178.68,129.67,153.52,155.95,163.20,169.46,202.46


In [455]:
dates_2020 = [col for col in apple_mobility.columns if col.startswith('2020')]

mobility_cols = ['transportation_type', 'sub-region'] + dates_2020
mobility_us = apple_mobility[apple_mobility['country'] == "United States"][mobility_cols]

mobility_us.rename(columns={'sub-region':'state'}, inplace=True)
mobility_us

mobility_us = mobility_us.melt(id_vars=["transportation_type", "state"], var_name="date", value_name="mobility")

# make new column based on transportation type
for value in ['driving', 'walking', 'transit']:
    mobility_us[f'mobility_{value}'] = np.where(mobility_us['transportation_type'] == value, mobility_us['mobility'].astype(float), float("NaN"))
    
mobility_us.drop(columns=['transportation_type', 'mobility'], inplace=True)

mobility_us_indexed = mobility_us.groupby(['state', 'date']).first()
mobility_us_indexed.reset_index(inplace=True, level=['date', 'state'])
mobility_us_indexed['date'] = pd.to_datetime(mobility_us_indexed['date'], infer_datetime_format=True)
mobility_us_indexed

,state,date,mobility_driving,mobility_walking,mobility_transit
0,Alabama,2020-01-13,100.00,100.00,100.00
1,Alabama,2020-01-14,100.39,94.69,105.69
2,Alabama,2020-01-15,101.95,98.19,102.70
3,Alabama,2020-01-16,105.97,102.98,104.55
4,Alabama,2020-01-17,124.99,122.60,114.79
...,...,...,...,...,...
18757,Wyoming,2020-12-27,180.01,NaN,NaN
18758,Wyoming,2020-12-28,173.78,NaN,NaN
18759,Wyoming,2020-12-29,176.76,NaN,NaN
18760,Wyoming,2020-12-30,179.88,NaN,NaN


In [456]:
# merge all dfs
mobility_by_state = blue_red.merge(mobility_us_indexed, left_on='stateName', right_on='state')\
    .drop(columns=['stateName', 'state'])\
    .rename(columns={'stateCode': 'state'})

# mobility_by_state
new_case_vs_mobility = new_cases.merge(mobility_by_state, on=['date', 'state'], how='inner')
final_data2 = new_case_vs_mobility.merge(pop_by_state, on='state', how='inner')
final_data2.replace(',','', regex=True, inplace=True)

In [445]:
final_data2.fillna(value=final_data2[f'mobility_{value}'].mean())

,date,state,new_case,mobility_driving,mobility_walking,mobility_transit,Population
0,2020-01-22,AK,0.0,104.94,99.90000,61.26529,734002
1,2020-01-23,AK,0.0,106.87,94.30000,61.26529,734002
2,2020-01-24,AK,0.0,123.88,120.33000,61.26529,734002
3,2020-01-25,AK,0.0,114.35,112.05000,61.26529,734002
4,2020-01-26,AK,0.0,81.75,80.32000,61.26529,734002
...,...,...,...,...,...,...,...
17245,2020-12-27,WY,54.0,180.01,61.26529,61.26529,567025
17246,2020-12-28,WY,504.0,173.78,61.26529,61.26529,567025
17247,2020-12-29,WY,219.0,176.76,61.26529,61.26529,567025
17248,2020-12-30,WY,210.0,179.88,61.26529,61.26529,567025


In [451]:
for value in ['driving', 'walking', 'transit']:
    final_data2[f'mobility_{value}'] = final_data2.fillna(value=final_data2[f'mobility_{value}'].mean())[f'mobility_{value}']

In [452]:
# final_data2.groupby("date").transform(lambda x: x.fillna(x.mean()))
final_data2

,date,state,new_case,mobility_driving,mobility_walking,mobility_transit,Population
0,2020-01-22,AK,0.0,104.94,99.900000,61.26529,734002
1,2020-01-23,AK,0.0,106.87,94.300000,61.26529,734002
2,2020-01-24,AK,0.0,123.88,120.330000,61.26529,734002
3,2020-01-25,AK,0.0,114.35,112.050000,61.26529,734002
4,2020-01-26,AK,0.0,81.75,80.320000,61.26529,734002
...,...,...,...,...,...,...,...
17245,2020-12-27,WY,54.0,180.01,126.892813,61.26529,567025
17246,2020-12-28,WY,504.0,173.78,126.892813,61.26529,567025
17247,2020-12-29,WY,219.0,176.76,126.892813,61.26529,567025
17248,2020-12-30,WY,210.0,179.88,126.892813,61.26529,567025


In [454]:
final_data2.corr()

,new_case,mobility_driving,mobility_walking,mobility_transit,Population
new_case,1.000000,-0.065020,0.041140,-0.128829,0.482867
mobility_driving,-0.065020,1.000000,0.690807,0.289863,-0.127217
mobility_walking,0.041140,0.690807,1.000000,0.339489,-0.008260
mobility_transit,-0.128829,0.289863,0.339489,1.000000,0.020798
Population,0.482867,-0.127217,-0.008260,0.020798,1.000000
